<a href="https://colab.research.google.com/github/maryembenghouma/Data-science-Projects/blob/main/Copie_deep_mimi_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and Initialize

In [ ]:
%%bash
pip install nltk
pip install datasets
pip install transformers[torch]
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub
pip install peft
pip install sentence_transformers
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4b02e2153d0eb202df91f35b9a7cacc1d6349bd1d888dabbceea4fba9aab9cc0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef179

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Import necessary libraries
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from peft import LoraConfig, get_peft_model, PromptTuningConfig
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from peft import TaskType
import faiss

**DATA EXPLORATION**

In this section of the code, we are performing an initial exploration of the dataset to understand its structure, contents, and basic statistics. This is an important step in any data science project as it helps identify the quality of the data and guides further preprocessing steps.

In [ ]:
# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")  # Use a small subset for testing
texts = dataset["article"]
summaries = dataset["highlights"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# Display the shape of the dataframe
dataset.shape

(2871, 3)

In [ ]:


# Set up the model and tokenizer
MODEL_NAME = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Using device: cuda


# Data Preparation

In [ ]:


# Split the data into training and validation sets
train_texts, val_texts, train_summaries, val_summaries = train_test_split(texts, summaries, test_size=0.1)
print(f"Training samples: {len(train_texts)}, Validation samples: {len(val_texts)}")


Training samples: 2583, Validation samples: 288


# Baseline with Prompt Engineering

In [ ]:
# Define the prompt format
PROMPT_FORMAT = "summarize: {}"

# Baseline function using simple prompt engineering
def generate_baseline_summary(text):
    input_text = PROMPT_FORMAT.format(text)
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    with torch.no_grad():
        summary_ids = model.generate(inputs.input_ids, max_length=100, num_beams=4, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Test the baseline model
sample_text = val_texts[0]
print("Original Text:", sample_text)
print("Baseline Generated Summary:", generate_baseline_summary(sample_text))


Original Text: (CNN) -- The daughter of Sen. Dick Durbin died Saturday from a congenital heart condition, the Illinois Democrat's office said. "Chris Durbin, 40, fought a heroic lifelong battle with heart disease and our thoughts and prayers are with the entire Durbin family," said Durbin's spokesman, Joe Shoemaker. Chris Durbin worked for 16 years for the U.S. Department of Agriculture in Washington. Survivors include her husband, Marty Johnson, and son, Alex. Her death comes as Dick Durbin, the second-ranking Democrat in the Senate, seeks a third term in Tuesday's general election. Funeral arrangements for his daughter were not immediately announced.
Baseline Generated Summary: Dick Durbin, a Democrat, is seeking a third term in the U.S. Senate.


# Prompt Tuning with PEFT

In [ ]:
# EarlyStopping class (same as before)
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.01):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
# Configure Prompt Tuning
prompt_tuning_config = PromptTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    num_virtual_tokens=20
)
model_prompt_tuning = get_peft_model(model, prompt_tuning_config)

early_stopping = EarlyStopping(patience=3, min_delta=0.01)

# Training function for prompt tuning
optimizer = torch.optim.AdamW(model_prompt_tuning.parameters(), lr=1e-4)

def train_prompt_tuning(texts, summaries):
    model_prompt_tuning.train()
    total_loss = 0
    for text, summary in zip(texts, summaries):
        input_text = PROMPT_FORMAT.format(text)
        inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length").to(device)
        labels = tokenizer(summary, return_tensors="pt", max_length=128, truncation=True, padding="max_length").to(device)

        outputs = model_prompt_tuning(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask, labels=labels.input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(texts)
    print(f"Training Loss: {avg_loss}")
    return avg_loss

# Fine-tune with early stopping
for epoch in range(3):  # Increase if desired; early stopping will halt training if needed
    print(f"Epoch {epoch+1}")
    avg_train_loss = train_prompt_tuning(train_texts, train_summaries)
    print()
    # Check early stopping condition on average training loss
    early_stopping(avg_train_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered. Stopping prompt tuning.")
        break

# Test the prompt-tuned model
sample_text = val_texts[0]
print("Prompt Tuning Generated Summary:", generate_baseline_summary(sample_text))

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1
Training Loss: 27.151942673275926

Epoch 2
Training Loss: 27.146438741148604

Epoch 3
Training Loss: 27.168978656025946

Prompt Tuning Generated Summary: Dick Durbin's daughter, Marty Johnson, has been diagnosed with a congenital heart condition.


# PEFT using LoRA

In [ ]:
from transformers import T5ForConditionalGeneration

# Initialize your model
model_lora = T5ForConditionalGeneration.from_pretrained("t5-small")
model_lora.to(device)  # Move to the correct device if necessary


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:

# Initialize early stopping
early_stopping = EarlyStopping(patience=3, min_delta=0.01)

# Training function for LoRA
optimizer = torch.optim.AdamW(model_lora.parameters(), lr=1e-4)

def train_lora(texts, summaries):
    model_lora.train()
    total_loss = 0
    for text, summary in zip(texts, summaries):
        input_text = PROMPT_FORMAT.format(text)
        inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length").to(device)
        labels = tokenizer(summary, return_tensors="pt", max_length=128, truncation=True, padding="max_length").to(device)

        outputs = model_lora(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask, labels=labels.input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(texts)
    print(f"Training Loss: {avg_loss}")
    return avg_loss

# Fine-tune with early stopping
for epoch in range(3):  # Increase if desired; early stopping will halt training if needed
    print(f"Epoch {epoch+1}")
    avg_train_loss = train_lora(train_texts, train_summaries)

    # Check early stopping condition on average training loss
    early_stopping(avg_train_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered. Stopping LoRA tuning.")
        break

# Test the LoRA-tuned model
sample_text = val_texts[0]
print("LoRA Generated Summary:", generate_baseline_summary(sample_text))

Epoch 1
Training Loss: 1.175475448462283
Epoch 2
Training Loss: 1.02661481412561
Epoch 3
Training Loss: 0.9567244900516702
LoRA Generated Summary: Durbin's daughter, Alex Durbin, died of a congenital heart condition.


In [ ]:
!pip install rouge-score

# **Initialize the ROUGE scorer**

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
def evaluate_model(model_name, generation_function, texts, references):
    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []

    for text, reference in zip(texts, references):
        generated_summary = generation_function(text)

        # Compute ROUGE scores
        scores = scorer.score(reference, generated_summary)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    # Calculate average scores
    avg_rouge1 = np.mean(rouge1_scores)
    avg_rouge2 = np.mean(rouge2_scores)
    avg_rougeL = np.mean(rougeL_scores)

    print(f"\n{model_name} Evaluation:")
    print(f"ROUGE-1: {avg_rouge1:.4f}")
    print(f"ROUGE-2: {avg_rouge2:.4f}")
    print(f"ROUGE-L: {avg_rougeL:.4f}")

    return avg_rouge1, avg_rouge2, avg_rougeL

In [ ]:
import numpy as np
evaluate_model("Baseline", generate_baseline_summary, val_texts, val_summaries)


Baseline Evaluation:
ROUGE-1: 0.2132
ROUGE-2: 0.0790
ROUGE-L: 0.1649


(0.2132239930855334, 0.07903650839707195, 0.1648853760837627)

In [ ]:
evaluate_model("Prompt Tuning", generate_baseline_summary, val_texts, val_summaries)



Prompt Tuning Evaluation:
ROUGE-1: 0.2207
ROUGE-2: 0.0822
ROUGE-L: 0.1703


(0.220719082457979, 0.08223653576016807, 0.1702892728321698)

In [ ]:
evaluate_model("LoRA", generate_baseline_summary, val_texts, val_summaries)



LoRA Evaluation:
ROUGE-1: 0.2107
ROUGE-2: 0.0753
ROUGE-L: 0.1642


(0.21067952170232196, 0.0752697886841049, 0.16421254488584303)